In [12]:
pip install python3-discogs-client

In [13]:
import discogs_client

In [14]:
# Replace the following user string and user_token first
# https://python3-discogs-client.readthedocs.io/en/latest/authentication.html
# ^ Check it and register an account on Discogs.com
d = discogs_client.Client('Your arbitrary user string here', user_token='your user token here')

In [15]:
class MyTrack:
    """
    One Instance represents one row in the final excel table.
    """
    def __init__(self, title, album, posi):
        self.title = title
        self.track_artist = set()
        self.album = album
        self.labels = set()
        self.release_year = None
        self.instruments = set()
        self.genre = set()
        self.format = set()
        self.countries_of_release = set()
        self.catalog_numbers = set()
        self.requires_manual_check = False
        self.id_str = title + " in " + album
        self.url = None
        self.position = posi
        self.track_specific_info = set()
        self.release_id = None

    def __str__ (self):
        return self.title + ' ' + self.album + ' ' + str(self.release_year)

    def __repr__ (self):
        return self.title + ' ' + self.album + ' ' + str(self.release_year)

In [16]:
class MyAlbum:
    def __init__(self, title):
        self.album_title = title
        self.track_specific_info = set()
        self.tracklist = []
        self.version_ids = set() #numbers
        self.has_track_specific_info_in_release_credits = False

    def __str__ (self):
        return self.album_title

    def __repr__ (self):
        return self.album_title

In [17]:
# 108475 is the artist id for Greg Phillinganes.
greg = d.artist(108475)
# All the releases/masters related to Greg Phillinganes.
greg_rel = greg.releases
# The release list can be paginated
greg_rel_page0 = greg_rel.page(0)
greg_rel_page5 = greg_rel.page(5)


In [71]:
# Final results will be stored in this dictionary
# Key: Each track's title
# Value: a MyTrack obj
# track_dic = {}
# Name to be searched
artist_name = greg.name
# The artist instance
artist_obj = greg

# List of albums
album_list = []

added_track_set = set()


def create_new_mytrack(cur_release, track_entry, is_release_main_artist, is_in_credits, main_release_name, cur_album):
    """
    Create a new MyTrack instance based on the track_entry in the cur_release.
    :param cur_album:
    :param cur_release: current version of the master
    :param track_entry: current track to create a new MyTrack instance
    :param is_release_main_artist: whether artist_obj is the main artist of cur_release
    :param is_in_credits: whether artist_name appears in the "Credits" part or not
    :param main_release_name: the title of the main_release of the master
    :return: title of the new track
    """
    if main_release_name == "":
        main_release_name = cur_release.title.strip()
    track_tmp = MyTrack(track_entry.title, main_release_name, track_entry.position)
    added_track_set.add(track_entry.title + " in " + main_release_name)
    track_tmp.url = cur_release.url
    track_tmp.release_id = cur_release.id
    for artist_entry in cur_release.artists:
        track_tmp.track_artist.add(artist_entry.name)
    for label_entry in cur_release.labels:
        track_tmp.labels.add(label_entry.name)
        track_tmp.catalog_numbers.add(label_entry.data['catno'])

    track_tmp.release_year = cur_release.year
    if is_release_main_artist:
        track_tmp.instruments.add("Main")
    if is_in_credits or artist_obj in track_entry.credits:
        for extra_artist_entry in cur_release.data['extraartists']:
            if extra_artist_entry['name'] == artist_name:
                track_tmp.instruments.add(extra_artist_entry['role'])
                if extra_artist_entry['tracks'] != "":
                    track_tmp.requires_manual_check = True
                    track_tmp.track_specific_info.add(extra_artist_entry['role'] + " -> tracks: " + extra_artist_entry['tracks'])
                    cur_album.track_specific_info.add("in releaseID: " + str(cur_release.id) + ": " + extra_artist_entry['role'] + " -> tracks: " + extra_artist_entry['tracks'])
                    cur_album.has_track_specific_info_in_release_credits = True
    track_tmp.genre.update(cur_release.genres)
    for format_entry in cur_release.formats:
        track_tmp.format.add(format_entry['name'])
    track_tmp.countries_of_release.add(cur_release.country)
    track_tmp.duration = track_entry.duration
    cur_album.tracklist.append(track_tmp)
    return track_tmp


def check_single_track_credit(cur_release, track_entry, artist_t, is_release_main_artist, is_in_credits, main_release_name, cur_album):
    """
    If the artist_obj is not the main artist of the release, then check artist_obj's contribution in a single track.
    :param cur_album:
    :param cur_release: current version of the master
    :param track_entry: current track to create a new MyTrack instance
    :param artist_t: artist instance to be found
    :param is_release_main_artist: whether artist_obj is the main artist of cur_release
    :param is_in_credits: whether artist_name appears in the "Credits" part or not
    :param main_release_name: the title of the main_release of the master
    :return: title of the current track, whether updated
    """
    if artist_t in track_entry.credits:
        cur_track_obj = create_new_mytrack(cur_release, track_entry, is_release_main_artist, is_in_credits, main_release_name, cur_album)
        for ex_artist_entry in track_entry.data['extraartists']:
            if ex_artist_entry['name'] == artist_name:
                new_role = ex_artist_entry['role']
                cur_track_obj.instruments.add(new_role)
        return cur_track_obj, True
    return None, False


def get_info_by_single_track(cur_release, is_release_main_artist, is_in_credits, main_release_name, cur_album):
    """
    If the artist_obj is not the main artist of the release, then check artist_obj's contribution track by track.
    :param cur_album:
    :param cur_release: current version of the master
    :param is_release_main_artist: whether artist_obj is the main artist of cur_release
    :param is_in_credits: whether artist_name appears in the "Credits" part or not
    :param main_release_name: the title of the main_release of the master
    :return: null
    """
    for track_e in cur_release.tracklist:
        check_single_track_credit(cur_release, track_e, artist_obj, is_release_main_artist, is_in_credits, main_release_name, cur_album)



def get_info_from_release(cur_release, main_release_name):
    """
    Complete data retrieving in one version/release.
    :param cur_release: current version of the master
    :param main_release_name: the title of the main_release of the master
    :return: list of created MyTrack instances' titles, whether current artist is the main artist || current artist appear in "Credits"
    """
    is_release_main_artist = False
    is_in_credits = False
    cur_album = MyAlbum(cur_release.title.strip())
    cur_album.version_ids.add(cur_release.id)
    album_list.append(cur_album)
    for main_artist_e in cur_release.artists:
        if main_artist_e.id == artist_id:
            is_release_main_artist = True
            break
    for credit_artist_e in cur_release.credits:
        if credit_artist_e.id == artist_id:
            is_in_credits = True
            break
    if is_release_main_artist or is_in_credits:
        for track_entry in cur_release.tracklist:
            create_new_mytrack(main_release_tmp, track_entry, is_release_main_artist, is_in_credits, main_release_name, cur_album)
        return cur_album, True
    else:
        get_info_by_single_track(cur_release, is_release_main_artist, is_in_credits, main_release_name, cur_album)
        return cur_album, False

def update_from_other_release_version(cur_release, updated_all_in_main_release, main_release_name, cur_album):
    """
    Update MyTrack in versions which are not main_release/version.
    :param cur_album:
    :param cur_release: current version of the master
    :param updated_all_in_main_release: in main_release of the master, whether current artist is the main artist || current artist appear in "Credits"
    :param main_release_name: the title of the main_release of the master
    :return: null
    """
    if cur_release.id in cur_album.version_ids:
        return
    else:
        cur_album.version_ids.add(cur_release.id)
    is_release_main_artist = False
    is_in_credits = False

    has_track_specific_info_in_credits = False
    for main_artist_e in cur_release.artists:
        if main_artist_e.id == artist_id:
            is_release_main_artist = True
            break
    for credit_artist_e in cur_release.credits:
        if credit_artist_e.id == artist_id:
            is_in_credits = True
            break
    if is_in_credits:        
        for extra_artist_entry in cur_release.data['extraartists']:
            if extra_artist_entry['name'] == artist_name and extra_artist_entry['tracks'] != "":
                new_str_id = extra_artist_entry['role'] + " -> tracks: " + extra_artist_entry['tracks']
                if not new_str_id in cur_album.track_specific_info:
                    cur_album.track_specific_info.add(new_str_id)
                    has_track_specific_info_in_credits = True
    update_all_cur_version = is_release_main_artist or is_in_credits

    if has_track_specific_info_in_credits or ( update_all_cur_version and not updated_all_in_main_release ):
        for track_entry in cur_release.tracklist:
            create_new_mytrack(cur_release, track_entry, is_release_main_artist, is_in_credits, main_release_name, cur_album)
    elif updated_all_in_main_release:
        return
    else:
        for track_entry in cur_release.tracklist:
            cur_str = track_entry.title + " in " + main_release_name
            if not cur_str in added_track_set:
                check_single_track_credit(cur_release, track_entry, artist_obj, is_release_main_artist, is_in_credits, main_release_name, cur_album)

# Main program starts here
for cur_rel in greg_rel_page1:
    # Each release's type is other master or release
    if cur_rel.data["type"] == "master":
        main_release_tmp = cur_rel.main_release
        # We first check the main_release of the master
        tmp_album, should_update_all = get_info_from_release(main_release_tmp, main_release_tmp.title.strip())
        # Then we iterate through other versions, mainly to collect new Countries of Release, Catalog #, Year,
        tmp_versions = cur_rel.versions
        for version_entry in tmp_versions:
            update_from_other_release_version(version_entry, should_update_all, main_release_tmp.title.strip(), tmp_album)
    else:
        tmp_album, boolean_never_used = get_info_from_release(cur_rel, cur_rel.title.strip())
    tmp_album.tracklist.sort(key=lambda e: (e.title, e.release_year, e.release_id))
    print(tmp_album.tracklist)


[Girl Talk Girl Talk 1981, Girl Talk Girl Talk 1981]
[Baby, I Do Love You Baby, I Do Love You 1981, Baby, I Do Love You Baby, I Do Love You 1981]
[Takin' It Up All Night Takin' It Up All Night 1981, Takin' It Up All Night Takin' It Up All Night 1981]
[Baby I Do Love You Significant Gains 1981, Big Man Significant Gains 1981, Do It All For Love Significant Gains 1981, Forever Now Significant Gains 1981, Girl Talk Significant Gains 1981, I Dont Want To Be The One Significant Gains 1981, Maxxed Out Significant Gains 1981, Takin It Up All Night Significant Gains 1981, The Call Significant Gains 1981]
[Behind The Mask Pulse 1985, Come As You Are Pulse 1985, Countdown To Love Pulse 1985, I Have Dreamed Pulse 1985, Lazy Nina Pulse 1985, Playin' With Fire Pulse 1985, Shake It Pulse 1985, Signals Pulse 1985, Won't Be Long Now Pulse 1985]
[Lazy Nina Lazy Nina 1985, Only You Lazy Nina 1985]
[Behind The Mask Countdown To Love / Behind The Mask 1985, Countdown To Love Countdown To Love / Behind The

[Ballada Populistyczna Jest 2004, Ballada Populistyczna Jest 2015, Breathing You Jest 2004, Breathing You Jest 2015, Dancing With Ghosts Jest 2004, Dancing With Ghosts Jest 2015, Doganiaj Jest 2004, Doganiaj Jest 2015, Każdy Ma Swoją Tajemnicę Jest 2004, Każdy Ma Swoją Tajemnicę Jest 2015, Nad Brzegiem Tęczy Jest 2004, Nad Brzegiem Tęczy Jest 2015, Niewidziani Jest 2004, Niewidziani Jest 2015, Skazani Na Ten Czas Jest 2004, Skazani Na Ten Czas Jest 2015, Spóźniamy Się Jest 2004, Spóźniamy Się Jest 2015, Tajemnicza Siła Jest 2004, Tajemnicza Siła Jest 2015, Taniec Cieni Jest 2004, Taniec Cieni Jest 2015, To Moja Pieśń Jest 2004, To Moja Pieśń Jest 2015, Trochę Słabej Woli Jest 2004, Trochę Słabej Woli Jest 2015, Wyścig Szczurów Jest 2004, Wyścig Szczurów Jest 2015, Za Duża Konkurencja Jest 2004, Za Duża Konkurencja Jest 2015, Śnieżna Kula Jest 2004, Śnieżna Kula Jest 2015]
[ABC The Ultimate Collection 2004, Bad The Ultimate Collection 2004, Beat It The Ultimate Collection 2004, Beat It 

[All Night Dancin' Triumph / Destiny 2009, All Night Dancin' Triumph / Destiny 2010, Blame It On The Boogie Triumph / Destiny 2009, Blame It On The Boogie Triumph / Destiny 2010, Blame It On The Boogie (John Luongo Disco Mix) Triumph / Destiny 2010, Bless His Soul Triumph / Destiny 2009, Bless His Soul Triumph / Destiny 2010, Can You Feel It Triumph / Destiny 2009, Can You Feel It Triumph / Destiny 2010, Destiny Triumph / Destiny 2009, Destiny Triumph / Destiny 2010, Destiny Triumph / Destiny 2010, Everybody Triumph / Destiny 2009, Everybody Triumph / Destiny 2010, Give It Up Triumph / Destiny 2009, Give It Up Triumph / Destiny 2010, Lovely One Triumph / Destiny 2009, Lovely One Triumph / Destiny 2010, Push Me Away Triumph / Destiny 2009, Push Me Away Triumph / Destiny 2010, Shake Your Body (Down To The Ground) Triumph / Destiny 2009, Shake Your Body (Down To The Ground) Triumph / Destiny 2010, Shake Your Body (Down To The Ground) (John Luongo Disco Mix) Triumph / Destiny 2010, That's 

[A Quien Llamas = Who Do You Call The Gap Band II 1980, Eres Mi Locura = You Are My High The Gap Band II 1980, I Don't Believe You Want To Get Up And Dance (Oops!) The Gap Band II 1979, I Don't Believe You Want To Get Up And Dance (Oops!) The Gap Band II 1979, I Don't Believe You Want To Get Up And Dance (Oops!) The Gap Band II 1979, I Don't Believe You Want To Get Up And Dance (Oops!) The Gap Band II 1979, Luces De Fiesta = Party Lights The Gap Band II 1980, Ningun Escondite = No Hiding Place The Gap Band II 1980, No Creo Que Quieras Salir A Bailar (Oops)= I Don't Believe You Want To Get Up And Dance (Oops!) The Gap Band II 1980, No Hiding Place The Gap Band II 1979, No Hiding Place The Gap Band II 1979, No Hiding Place The Gap Band II 1979, No Hiding Place The Gap Band II 1979, Party Lights The Gap Band II 1979, Party Lights The Gap Band II 1979, Party Lights The Gap Band II 1979, Party Lights The Gap Band II 1979, Pisando (Hacia Afuera) = Steppin' (Out) The Gap Band II 1980, Steppin

[At Last My Search Is Over At Last... The Pips 1977, Happiness At Last... The Pips 1977, If I Could Bring Back Yesterday At Last... The Pips 1977, Midnight Flight To Your Love At Last... The Pips 1977, Since I Found Love At Last... The Pips 1977, Tomorrow Child At Last... The Pips 1977, Uncle James At Last... The Pips 1977]
[Bye, Bye, Baby When I Get Back Home 1977, Ghetto Penthouse When I Get Back Home 1977, Good To Me When I Get Back Home 1977, Leave Your Spirit Behind When I Get Back Home 1977, Leaving You Is Killing Me When I Get Back Home 1977, Mr. Sin When I Get Back Home 1977, Paradise When I Get Back Home 1977, Sexasonic When I Get Back Home 1977, To Make You Stay When I Get Back Home 1977, When I Get Back Home When I Get Back Home 1977]
[Don't Overlook The Feelings Sunbear 1977, Don't Overlook The Feelings Sunbear 1977, Erika Sunbear 1977, Erika Sunbear 1977, Fantasy Sunbear 1977, Fantasy Sunbear 1977, I Heard The Voice Of Music Say Sunbear 1977, I Heard The Voice Of Music Say

In [72]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [73]:
def convert_tracks_dic_to_df(album_list):
    all_tracks = []
    for album in album_list:
        for track in album.tracklist:
            
            cur_track = {
            'album_title' : album.album_title,
            'track_specific_info' : ', '.join(album.track_specific_info),
            'version_ids' : ', '.join(str(v) for v in album.version_ids),
            'has_track_specific_info_in_release_credits' : album.has_track_specific_info_in_release_credits
            }
            
            track_dic = track.__dict__
            
            for i in track_dic:
                if type(track_dic[i]) is set:              # convert set to string, split by ','
                    cur_track[i] = ', '.join(str(v) for v in track_dic[i])
                else:
                    cur_track[i] = track_dic[i]
                    
            all_tracks.append(cur_track)
    df = pd.DataFrame(all_tracks)
    # replace empty cells to NAN
    df = df.replace('', np.NaN)
    return df

In [74]:
df = convert_tracks_dic_to_df(album_list)
df

,album_title,track_specific_info,version_ids,has_track_specific_info_in_release_credits,title,track_artist,album,labels,release_year,instruments,genre,format,countries_of_release,catalog_numbers,requires_manual_check,id_str,url,position,release_id,duration
0,Girl Talk,NaN,"21792616, 10245075",False,Girl Talk,Greg Phillinganes,Girl Talk,Planet (15),1981,"Producer, Main","Funk / Soul, Electronic",Vinyl,US,P-47938,False,Girl Talk in Girl Talk,https://www.discogs.com/release/10245075-Greg-...,A,10245075,3:43
1,Girl Talk,NaN,"21792616, 10245075",False,Girl Talk,Greg Phillinganes,Girl Talk,Planet (15),1981,"Producer, Main","Funk / Soul, Electronic",Vinyl,US,P-47938,False,Girl Talk in Girl Talk,https://www.discogs.com/release/10245075-Greg-...,B,10245075,3:43
2,"Baby, I Do Love You",NaN,"10694085, 648041, 3597653",False,"Baby, I Do Love You",Greg Phillinganes,"Baby, I Do Love You",Planet (15),1981,"Producer, Written-By, Main","Funk / Soul, Electronic",Vinyl,US,P-47928,False,"Baby, I Do Love You in Baby, I Do Love You",https://www.discogs.com/release/648041-Greg-Ph...,A,648041,4:15
3,"Baby, I Do Love You",NaN,"10694085, 648041, 3597653",False,"Baby, I Do Love You",Greg Phillinganes,"Baby, I Do Love You",Planet (15),1981,"Producer, Written-By, Main","Funk / Soul, Electronic",Vinyl,US,P-47928,False,"Baby, I Do Love You in Baby, I Do Love You",https://www.discogs.com/release/648041-Greg-Ph...,AA,648041,4:15
4,Takin' It Up All Night,NaN,"7377370, 1998347, 21399574",False,Takin' It Up All Night,Greg Phillinganes,Takin' It Up All Night,Planet (15),1981,"Producer, Written-By, Main","Funk / Soul, Rock",Vinyl,US,AS-11514,False,Takin' It Up All Night in Takin' It Up All Night,https://www.discogs.com/release/1998347-Greg-P...,A,1998347,4:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,Maxi,NaN,"18916384, 4438910, 16221268, 3582264, 11174682...",False,Lover To Lover,Maxi Anderson,Maxi,Blue Note,1977,Keyboards,Funk / Soul,Vinyl,US,BN-LA738-H,False,Lover To Lover in Maxi,https://www.discogs.com/release/1009117-Maxi-Maxi,B2,1009117,4:16
998,Maxi,NaN,"18916384, 4438910, 16221268, 3582264, 11174682...",False,Music On My Mind,Maxi Anderson,Maxi,Blue Note,1977,Keyboards,Funk / Soul,Vinyl,US,BN-LA738-H,False,Music On My Mind in Maxi,https://www.discogs.com/release/1009117-Maxi-Maxi,B5,1009117,3:27
999,Maxi,NaN,"18916384, 4438910, 16221268, 3582264, 11174682...",False,The Perfect Day,Maxi Anderson,Maxi,Blue Note,1977,Keyboards,Funk / Soul,Vinyl,US,BN-LA738-H,False,The Perfect Day in Maxi,https://www.discogs.com/release/1009117-Maxi-Maxi,A4,1009117,3:28
1000,Maxi,NaN,"18916384, 4438910, 16221268, 3582264, 11174682...",False,This One's For You,Maxi Anderson,Maxi,Blue Note,1977,Keyboards,Funk / Soul,Vinyl,US,BN-LA738-H,False,This One's For You in Maxi,https://www.discogs.com/release/1009117-Maxi-Maxi,A5,1009117,4:34


In [75]:
# convert dataframe to csv file
import os  
os.makedirs('output', exist_ok=True)
df.to_csv('output/v2_sample_output_Greg_Phillinganes_page1.csv') 
df.to_excel("output/v2_sample_output_Greg_Phillinganes_page1.xlsx")